# Оценка кровяного давления

Задача состоит в том чтобы оценить систолическое (SBP) и диастолическое (DBP) кровяное давление для конкретного человека используя фотоплетизмограмму (PPG) и электрокардиограмму (ECG). Для каждого человека открыты несколько «калибровочных» измерений, то есть примерно 20% данных для каждого человека содержат запись о SBP и DBP наряду с соответствующими записями PPG и ECG.

Данные сигналов PPG и ECG записаны в файлы вида: subjXXlogYYYY.csv, где XX есть идентификационный номер человека, а YYYY — номер записи. Первая строка файла содержит значения SBP и DBP для калибровочных измерений и нули для остальных, а последующие строки содержат семплы PPG и ECG сигналы.

In [1]:
import pandas as pd
import numpy as np
import os
import scipy

In [28]:
work_dir = '../data/data_train/'

In [29]:
files_list = [f for f in os.listdir(work_dir) if os.path.isfile(work_dir+f)]

In [30]:
len(files_list) # общее количество файлов

894

Посмотрим на структуру файлов

In [31]:
data = pd.read_csv(work_dir + files_list[0])

In [32]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 2 columns):
126    15000 non-null int64
79     15000 non-null int64
dtypes: int64(2)
memory usage: 234.5 KB


SBP = 100, DBP = 67, следующие 15000 строк содержат PPG и ECG сигналы  

In [33]:
subjs = list(set([int(fl.split('log')[0].split('subj')[1]) for fl in files_list])) # id участников эксперимента

In [34]:
print(subjs)

[3, 7, 9, 11, 14, 16, 18, 21, 22, 26, 27, 30]


Метрика качества по условиям задачи определяется как взвешенная сумма корней среднеквадратичных ошибок (RMSE), где ошибка определения DBP имеет в 2 раза больший вес, чем ошибка SBP

In [17]:
def loss_f(sbp, sbpt, dbp, dbpt):
    s, d = 0, 0
    n = len(dbp)
    for i in range(n):
        s += (int(round(sbp[i])) - sbpt[i])**2
        d += (int(round(dbp[i])) - dbpt[i])**2
    return s, d

Один из подходов к выделению признаков из сигналов PPG и ECG состоит в том, чтобы использовать коэффициенты БПФ. Для ускорения работы будем записывать в файл только первые 1000 коэффициентов. Так как измерения PPG и ECG независимы, можно использовать только один вид сигнала, в данном случае ECG. Из условий задачи следует, что электроды ECG могут быть наложены уже после начала записи. Поэтому несколько первых записей отброшены

In [18]:
def write_fft(subj_files):
    subj_train_files, subj_test_files = [], []
    for sf in subj_files:
        data = pd.read_csv(work_dir + sf)
        if int(data.columns[0]):
            subj_train_files.append(sf)
        else:
            subj_test_files.append(sf)

        X = data[data.columns[1]].iloc[10:]
        fft_feat = abs(scipy.fft(X)[:1000])
        base_fn, ext = os.path.splitext(sf)
        data_fn = work_dir + 'fft/' + base_fn + '.fft'
        scipy.save(data_fn, fft_feat)
    return subj_train_files, subj_test_files

def read_fft(files):
    X, y0, y1 = [],[],[]
    for tf in files:
        data = pd.read_csv(work_dir + tf)
        y0.append(round(float(data.columns[0])))
        y1.append(round(float(data.columns[1])))
        base_fn, ext = os.path.splitext(tf)
        data_fn = work_dir + 'fft/' + base_fn + '.fft.npy'
        fft_feat = scipy.load(data_fn)
        X.append(fft_feat[:1000])
    return np.array(X), np.array(y0), np.array(y1)

In [25]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn import linear_model

models = [linear_model.BayesianRidge(), LinearRegression(), RandomForestRegressor()]

In [35]:
for model in models:
    loss_s, loss_d = 0, 0

    for i in subjs:
        subj_files = [f for f in files_list if f.startswith('subj' + str(i) + 'log')]
        subj_train, subj_test = write_fft(subj_files)
        subj_train = subj_files[:int(len(subj_files)*0.2)]
        subj_test = subj_files[int(len(subj_files)*0.2):]
        X_train, y0_train, y1_train = read_fft(subj_train)
        X_test, y0_real, y1_real = read_fft(subj_test)
        model.fit(X_train, y0_train)
        y0_test = model.predict(X_test)
        model.fit(X_train, y1_train)
        y1_test = model.predict(X_test)
        s, d  = loss_f(y0_test, y0_real, y1_test, y1_real)
        loss_s += s
        loss_d += d 
    
    lf = len(files_list)
    print(model)    
    print(100. * (loss_s / float(lf))**0.5 + 200. * (loss_d / float(lf))**0.5)
    print(' ')

BayesianRidge(alpha_1=1e-06, alpha_2=1e-06, compute_score=False, copy_X=True,
       fit_intercept=True, lambda_1=1e-06, lambda_2=1e-06, n_iter=300,
       normalize=False, tol=0.001, verbose=False)
3778.112470053079
 
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)
4812.513151152262
 
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
2317.0599427451702
 


Из трех моделей лучше всего сработал RandomForest. Была идея попробовать также классификаторы (давление одного человека колеблется в небольшом диапазоне), но времени не хватило